In [2]:
from random import random , randint , choice
from copy import deepcopy
from math import log

In [3]:
class fwrapper:
    def __init__(self , function , childcount , name):
        self.function = function
        self.childcount = childcount
        self.name = name
    

class node:
    def __init__(self , fw , children):
        self.function = fw.function
        self.name = fw.name
        self.children = children
        
    def evaluate(self , inp):
        results = [n.evaluate(inp) for n in self.children]
        
        return self.function(results)
    
    def display(self , indent = 0):
        print(' '*indent + self.name)
        
        for c in self.children:
            c.display(indent+1)


class paramnode:
    def __init__(self , idx):
        self.idx = idx
    
    def evaluate(self , inp):
        return inp[self.idx]
    
    def display(self , indent=0):
        print('%sp%d' % (' '*indent , self.idx))

    
class constnode:
    def __init__(self , v):
        self.v = v
    
    def evaluate(self , inp):
        return self.v
    
    def display(self , indent = 0):
        print('%s%d' % (' '*indent , self.v))

In [4]:
addw = fwrapper(lambda l: l[0]+l[1] , 2 , 'add')
subw = fwrapper(lambda l: l[0]-l[1] , 2 , 'subtract')
mulw = fwrapper(lambda l: l[0]-l[1] , 2 , 'multiply')

def iffunc(l):
    if l[0]>0:
        return l[1]
    else:
        return l[2]

ifw = fwrapper(iffunc , 3 , 'if')

def isgreater(l):
    if l[0]>l[1]:
        return 1
    else:
        return 0

gtw = fwrapper(isgreater , 2 , 'isgreater')

flist = [addw , mulw , ifw , gtw , subw]

In [5]:
def exampletree():
    return node(ifw , [
        node(gtw , [paramnode(0) , constnode(3)]),
        node(addw ,[paramnode(1) , constnode(5)]),
        node(subw ,[paramnode(1) , constnode(2)]),
    ])

In [5]:
exampletree_ = exampletree()

In [6]:
exampletree_.evaluate([2,3])

1

In [7]:
exampletree_.evaluate([5,3])

8

In [9]:
exampletree_ = exampletree()

In [10]:
exampletree_.display()

if
 isgreater
  p0
  3
 add
  p1
  5
 subtract
  p1
  2


In [6]:
def makerandomtree(pc , maxdepth=4 , fpr=0.5 , ppr = 0.6):
    if random()<fpr and maxdepth>0:
        f = choice(flist)
        
        children = [makerandomtree(pc , maxdepth-1 , fpr , ppr) 
                   for i in range(f.childcount)]
        
        return node(f , children)
    elif random()<ppr:
        return paramnode(randint(0,pc-1))
    else:
        return constnode(randint(0 , 10))

In [35]:
random1 = makerandomtree(2)

In [15]:
random1.evaluate([7,1])

1

In [36]:
random1.display()

8


In [7]:
#准备进行预测的函数
def hiddenfunction(x , y):
    return x**2+2*y+3*x+5

In [8]:
def buildhiddenset():
    rows = []
    
    for i in range(200):
        x = randint(0 , 40)
        y = randint(0 , 40)
        rows.append([x, y,hiddenfunction(x,y)])
    
    return rows

In [20]:
hiddenset = buildhiddenset()

In [21]:
hiddenset

[[33, 36, 1265],
 [20, 28, 521],
 [8, 27, 147],
 [30, 2, 999],
 [20, 8, 481],
 [34, 1, 1265],
 [37, 9, 1503],
 [20, 18, 501],
 [30, 30, 1055],
 [29, 22, 977],
 [27, 25, 865],
 [2, 10, 35],
 [6, 27, 113],
 [33, 13, 1219],
 [16, 6, 321],
 [6, 9, 77],
 [12, 26, 237],
 [10, 31, 197],
 [8, 17, 127],
 [9, 40, 193],
 [14, 40, 323],
 [39, 22, 1687],
 [3, 25, 73],
 [9, 38, 189],
 [10, 9, 153],
 [24, 10, 673],
 [16, 2, 313],
 [20, 34, 533],
 [37, 1, 1487],
 [10, 36, 207],
 [7, 24, 123],
 [39, 8, 1659],
 [18, 1, 385],
 [11, 6, 171],
 [31, 2, 1063],
 [7, 8, 91],
 [38, 35, 1633],
 [28, 17, 907],
 [25, 20, 745],
 [5, 14, 73],
 [31, 19, 1097],
 [5, 29, 103],
 [37, 20, 1525],
 [1, 36, 81],
 [32, 14, 1153],
 [20, 32, 529],
 [0, 24, 53],
 [27, 36, 887],
 [8, 28, 149],
 [22, 34, 623],
 [19, 5, 433],
 [34, 15, 1293],
 [8, 24, 141],
 [17, 1, 347],
 [28, 2, 877],
 [21, 14, 537],
 [29, 31, 995],
 [29, 40, 1013],
 [3, 23, 69],
 [13, 6, 225],
 [3, 25, 73],
 [35, 14, 1363],
 [37, 8, 1501],
 [30, 29, 1053],
 [18

In [9]:
#衡量程序的好坏
#相当于适应度函数
def scorefunction(tree , s):
    dif = 0
    
    for data in s:
        v = tree.evaluate([data[0] , data[1]])
        dif += abs(v-data[2])
    
    return dif

In [23]:
scorefunction(random1 , hiddenset)

126236

In [10]:
#变异
def mutate(t , pc , probchange = 0.1):
    if random()<probchange:
        return makerandomtree(pc)
    else:
        result = deepcopy(t)
        if isinstance(t , node):
            result.children = [mutate(c , pc , probchange) for c in t.children]
        
        return result

In [34]:
random1.display()

4


In [37]:
random1 = exampletree_

In [38]:
random1.display()

if
 isgreater
  p0
  3
 add
  p1
  5
 subtract
  p1
  2


In [39]:
muttree = mutate(random1 , 2)

In [40]:
muttree.display()

if
 isgreater
  p0
  3
 add
  isgreater
   2
   isgreater
    6
    subtract
     isgreater
      p0
      p1
     8
  0
 subtract
  p1
  2


In [41]:
scorefunction(random1 , hiddenset)

125348

In [42]:
scorefunction(muttree , hiddenset)

129886

In [11]:
#交叉
def crossover(t1 , t2 , probswap=0.7 , top=1):
    if random()<probswap and not top:
        return deepcopy(t2)
    else:
        result = deepcopy(t1)
        
        if isinstance(t1 , node) and isinstance(t2 , node):
            result.children = [crossover(c,choice(t2.children),probswap,0) 
                              for c in t1.children]
        
        return result

In [44]:
random2 = makerandomtree(2)

In [45]:
random2.display()

p0


In [46]:
cross = crossover(random1 , random2)

In [47]:
cross.display()

if
 isgreater
  p0
  3
 add
  p1
  5
 subtract
  p1
  2


In [48]:
scorefunction(cross , hiddenset)

125348

In [12]:
#开始进化
def evolve(pc , popsize , rankfunction , maxgen=500,
          mutationrate=0.1 , breedingrate=0.4 , pexp=0.7 , pnew=0.05):
    def selectindex():
        return int(log(random()) / log(pexp))
    
    population = [makerandomtree(pc) for i in range(popsize)]
    
    for i in range(maxgen):
        scores = rankfunction(population)
        
        print(scores[0][0])
        
        if scores[0][0] == 0:
            break
        
        newpop = [scores[0][1] , scores[1][1]]
        
        while len(newpop)<popsize:
            if random()>pnew:
                newpop.append(mutate(
                crossover(scores[selectindex()][1],
                         scores[selectindex()][1],
                         probswap = breedingrate),
                pc , probchange=mutationrate))
            else:
                newpop.append(makerandomtree(pc))
        
        population = newpop
    
    scores[0][1].display()
    
    return scores[0][1]

In [13]:
def getrankfunction(dataset):
    def rankfunction(population):
        scores = [(scorefunction(t , dataset) , t) for t in population]
        scores.sort()
        return scores
    
    return rankfunction


In [51]:
rf = getrankfunction(buildhiddenset())

In [ ]:
evolve(2 , 500 , rf , mutationrate=0.2 , breedingrate=0.1 , pexp=0.7 , pnew=0.1)

109937
104348
97898
92343
81398
80921
70843
66247
64505
60666
58422
58422
56956
56956
52470
52470
52470
52470
46549
46549
46549
46549
46549
46549
46549
46549
46549
43685
43685
43685
42609
42609
42609
37383
37383
37383
34997
34997
34997
34997
34997
34997
34997
34997
34997
34997
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551
33551


In [14]:
def gridgame(p):
    max_ = (3,3)
    
    lastmove=[-1,-1]
    location = [[randint(0,max_[0]) , randint(0,max_[1])]]
    
    location.append([(location[0][0] + 2) % 4 , (location[0][1]+2)%4])
    
    for o in range(50):
        for i in range(2):
            locs = location[i][:] + location[1-i][:]
            locs.append(lastmove[i])
            move = p[i].evaluate(locs)%4
            
            if lastmove[i] == move:
                return 1-i
            
            lastmove[i] = move
            
            if move==0:
                location[i][0] -= 1
                
                if location[i][0]<0:
                    location[i][0] = 0
            if move == 1:
                location[i][0] += 1
                if location[i][0]>max_[0]:
                    location[i][0] = max_[0]
            
            if move == 2:
                location[i][1] -= 1
                if location[i][1]<0:
                    location[i][1] = 0
            
            if move==3:
                location[i][1] += 1
                if location[i][1] > max_[1]:
                    location[i][1] = max_[1]
            
            
            if location[i] == location[1-i]:
                return i
    
    return -1

In [15]:
p1 = makerandomtree(5)
p2 = makerandomtree(5)

In [16]:
gridgame([p1 , p2])

1

In [17]:
def tournament(p1):
    losses = [0 for p in p1]
    
    for i in range(len(p1)):
        for j in range(len(p1)):
            if i == j:
                continue
            
            winner = gridgame([p1[i] , p1[j]])
            
            if winner == 0:
                losses[j] += 2
            elif winner == 1:
                losses[j] += 2
            elif winner == -1:
                losses[i] += 1
                losses[i] += 1
                pass
        
    z = zip(losses , p1)
    z.sort()
    
    return z

In [20]:
winner = evolve(5 , 100 , tournament , maxgen=50)

180
182
184
178
176
178
180
174
174
182
184
182
178
182
178
168
178
178
180
178
178
178
186
180
178
182
180
176
178
178
182
172
178
182
182
184
170
180
178
178
182
182
182
170
174
180
172
178
176
174
multiply
 multiply
  subtract
   subtract
    5
    p3
   p0
  p1
 p3


In [26]:
class humanplayer:
    def evaluate(self , board):
        me = tuple(board[0:2])
        others = [tuple(board[x:x+2]) for x in range(2 , len(board)-1 , 2)]
        
        for i in range(4):
            for j in range(4):
                if (i,j) == me:
                    print 'O'
                elif (i,j) in others:
                    print 'X'
                else:
                    print '.'
        
        print()
        
        print('your last move %d' % board[len(board) - 1 ])
        print(' 0')
        print('2  3')
        print(' 1')
        print('enter move:')
        
        move = int(raw_input())
        
        return move

In [27]:
gridgame([winner , humanplayer()])

.
.
.
.
.
.
.
X
.
.
.
.
.
O
.
.
()
your last move -1
 0
2  3
 1
enter move:
1


1